In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import spacy
from spacy.lang.en import English
spacy.load('en')
nlp = English()
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
# CONSTANTS
PUNCTS = string.punctuation and {'...', '?', '(', ')'} # TODO: should be improved
STOP_WORDS = set(stopwords.words("english")) - {'who', 'whom', 'what', 'when', 'where', 'why', 'how'}

In [72]:
df = pd.read_csv('train_dup.csv').drop(columns='id', axis=1)
df = df.dropna()
df.head()

,qid1,qid2,question1,question2,is_duplicate
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [73]:
duplicates = df[df.is_duplicate==1]
not_duplicates = df[df.is_duplicate == 0]
print(f'The proportion of classes pairs/not pairs is: {len(duplicates)}/{len(not_duplicates)}')
print(f'An example of duplicated questions: \n{duplicates.question1[5]} \nAND \n{duplicates.question2[5]}')
duplicates.head()

The proportion of classes pairs/not pairs is: 149263/255027
An example of duplicated questions: 
Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 
AND 
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


,qid1,qid2,question1,question2,is_duplicate
5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [33]:
# checking out from habr. This example will be later removed
def compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word, pos):
    """
    Print the results of stemmind and lemmitization using the passed stemmer, lemmatizer, word and pos (part of speech)
    """
    print("Stemmer:", stemmer.stem(word))
    print("Lemmatizer:", lemmatizer.lemmatize(word, pos))
    print()

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word = "seen", pos = wordnet.VERB)
compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word = "drove", pos = wordnet.VERB)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Stemmer: seen
Lemmatizer: see

Stemmer: drove
Lemmatizer: drive



In [37]:
# This is an example, will be later removed
count_vectorizer = CountVectorizer()
print(duplicates.question1[5], duplicates.question2[5])
# Create the Bag-of-Words Model
bag_of_words = count_vectorizer.fit_transform([duplicates.question1[5], duplicates.question2[5]])

# Show the Bag-of-Words Model as a pandas DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


,about,am,and,ascendant,astrology,cap,capricorn,does,in,me,moon,rising,say,sun,that,this,triple,what
0,1,1,1,0,1,2,1,1,0,1,1,1,1,1,1,0,0,1
1,1,0,1,1,0,0,2,1,1,1,1,0,1,1,0,1,1,1


Так, мы здесь взяли 2 похожих вопроса, каждый соответствует ряду теперь. Можно заметить, что вектора типа очень близки, но нужно смотреть на ключевые слова, а не на предлоги, союзы и т.д


Мы будем потом выбрасывать эти стоп-слова, которые есть в stopwords.words("english")

# Планы:

игнорирование регистра слов;


игнорирование пунктуации;


выкидывание стоп-слов;


приведение слов к их базовым формам (лемматизация);


исправление неправильно написанных слов.


подсчет н-грамм


TF/IDF - как помогут?


как преобразовать в числа?

![title](project_work_scheme.png)

In [63]:
# nltk cleaning: very slow!
lemmatizer = WordNetLemmatizer()
def clean_sentence_lemmatizer(sentence):
    """
    Receives a raw sentence and clean it using the following steps:  # BETTER
    1. Remove all non-words
    2. Transform the review in lower case
    3. Remove all stop words
    4. Perform lemmatizer

    Args:
        sentence: the sentence that will be cleaned
    Returns:
        a clean sentence using the mentioned steps above.
    """
    
    sentence = re.sub("[^A-Za-z]", " ", sentence)
    sentence = sentence.lower()
    sentence = word_tokenize(sentence)
    sentence = [lemmatizer.lemmatize(word) for word in sentence if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)
    return sentence

print(duplicates.question1[5])
clean_sentencelemm(duplicates.question1[5])

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?


'astrology capricorn sun cap moon cap rising say'

## SPACY

Tokenization, lemmatization, removing stop words (except question opening words like who, why, etc) and puctuation and lowering

In [104]:
clean_question = lambda sentence: ' '.join([word.lemma_.lower() for word in nlp(sentence) if word not in STOP_WORDS if word.lemma_ not in PUNCTS])
df_clean = df.copy()
df_clean['question1'] = df.apply(lambda row: clean_question(row['question1']), axis=1)
df_clean['question2'] = df.apply(lambda row: clean_question(row['question2']), axis=1)
df_clean.head()

,qid1,qid2,question1,question2,is_duplicate
0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,3,4,what is the story of kohinoor koh - i - noor d...,what would happen if the indian government sto...,0
2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,7,8,why am i mentally very lonely how can i solve it,find the remainder when [ math]23^{24}[/math ]...,0
4,9,10,"which one dissolve in water quikly sugar , sal...",which fish would survive in salt water,0


Some basic stats

In [127]:
unigramm_count_dict_initial = {}
bigramm_count_dict_initial = {}
unigramm_count_dict_clean = {}
bigramm_count_dict_clean = {}
number_of_words_inq_initial = []
number_of_words_inq_clean = []
lengths_of_questions_initial = []
lengths_of_questions_clean = []

# dirty df
for q1, q2 in zip(df.question1, df.question2):
    unigramms_q1 = [word.text for word in nlp(q1)]
    unigramms_q2 = [word.text for word in nlp(q2)]
    number_of_words_inq_initial.append(len(unigramms_q1))
    number_of_words_inq_initial.append(len(unigramms_q2))
    lengths_of_questions_initial.append(len(q1))
    lengths_of_questions_initial.append(len(q2))
    
    for unigramm in unigramms_q1 + unigramms_q2:
        if unigramm in unigramm_count_dict_initial:
            unigramm_count_dict_initial[unigramm] += 1
        else:
            unigramm_count_dict_initial[unigramm] = 1
            
    bigramms_q1 = [' '.join(q1.split()[i:i+2]) for i in range(len(q1.split())) if i < len(q1.split()) - 1]
    bigramms_q2 = [' '.join(q2.split()[i:i+2]) for i in range(len(q2.split())) if i < len(q2.split()) - 1]
    
    for bigramm in bigramms_q1 + bigramms_q2:
        if bigramm in bigramm_count_dict_initial:
            bigramm_count_dict_initial[bigramm] += 1
        else:
            bigramm_count_dict_initial[bigramm] = 1
            
# clean df            
for q1, q2 in zip(df_clean.question1, df_clean.question2):
    unigramms_q1 = [word.text for word in nlp(q1)]
    unigramms_q2 = [word.text for word in nlp(q2)]
    number_of_words_inq_clean.append(len(unigramms_q1))
    number_of_words_inq_clean.append(len(unigramms_q2))
    lengths_of_questions_clean.append(len(q1))
    lengths_of_questions_clean.append(len(q2))
    
    for unigramm in unigramms_q1 + unigramms_q2:
        if unigramm in unigramm_count_dict_initial:
            unigramm_count_dict_initial[unigramm] += 1
        else:
            unigramm_count_dict_initial[unigramm] = 1
            
    bigramms_q1 = [' '.join(q1.split()[i:i+2]) for i in range(len(q1.split())) if i < len(q1.split()) - 1]
    bigramms_q2 = [' '.join(q2.split()[i:i+2]) for i in range(len(q2.split())) if i < len(q2.split()) - 1]
    
    for bigramm in bigramms_q1 + bigramms_q2:
        if bigramm in bigramm_count_dict_initial:
            bigramm_count_dict_initial[bigramm] += 1
        else:
            bigramm_count_dict_initial[bigramm] = 1


In [126]:
bigramm_count_dict_initial

{'What is': 114500,
 'is the': 201427,
 'the step': 115,
 'step by': 225,
 'by step': 164,
 'step guide': 21,
 'guide to': 97,
 'to invest': 1849,
 'invest in': 1680,
 'in share': 89,
 'share market': 183,
 'market in': 198,
 'in india?': 865,
 'share market?': 89,
 'the story': 597,
 'story of': 388,
 'of Kohinoor': 4,
 'Kohinoor (Koh-i-Noor)': 6,
 '(Koh-i-Noor) Diamond?': 4,
 'What would': 5237,
 'would happen': 4028,
 'happen if': 4924,
 'if the': 3865,
 'the Indian': 2489,
 'Indian government': 501,
 'government stole': 4,
 'stole the': 16,
 'the Kohinoor': 5,
 '(Koh-i-Noor) diamond': 2,
 'diamond back?': 7,
 'How can': 53008,
 'can I': 54095,
 'I increase': 1886,
 'increase the': 1293,
 'the speed': 1375,
 'speed of': 1662,
 'of my': 6164,
 'my internet': 94,
 'internet connection': 103,
 'connection while': 2,
 'while using': 89,
 'using a': 1255,
 'a VPN?': 25,
 'can Internet': 2,
 'Internet speed': 24,
 'speed be': 15,
 'be increased': 92,
 'increased by': 42,
 'by hacking': 3,